In [9]:
import os
import shutil

In [10]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [20]:
shutil.rmtree('/content/drive/MyDrive/splitted')
# force remove

In [ ]:
shutil.rmtree('/content/drive/MyDrive/dataset')

In [ ]:
!unzip '/content/drive/MyDrive/dataset.zip' -d '/content/drive/MyDrive/dataset'

In [21]:
original_dataset_dir='/content/drive/MyDrive/dataset'
#
classes_list=os.listdir(original_dataset_dir)

base_dir='/content/drive/MyDrive/splitted'
os.mkdir(base_dir)

#path.join join one or more path components intelligently. 
#This method concatenates various path components with exactly one directory separator 
#(‘/’) following each non-empty part except the last path component. 
train_dir=os.path.join(base_dir,'train') 
os.mkdir(train_dir)
validation_dir=os.path.join(base_dir,'val')
os.mkdir(validation_dir)
test_dir=os.path.join(base_dir,'test')
os.mkdir(test_dir)

for clss in classes_list:
  os.mkdir(os.path.join(train_dir,clss))
  os.mkdir(os.path.join(validation_dir,clss))
  os.mkdir(os.path.join(test_dir,clss))

In [22]:
import math
 
for cls in classes_list:
    path = os.path.join(original_dataset_dir, cls)
    fnames = os.listdir(path)
 
    train_size = math.floor(len(fnames) * 0.6)
    validation_size = math.floor(len(fnames) * 0.2)
    test_size = math.floor(len(fnames) * 0.2)
    

    #split dataset into train, validation, and test set
    train_fnames = fnames[:train_size]
    print("Train size(",cls,"): ", len(train_fnames))
    for fname in train_fnames:
        src = os.path.join(path, fname)
        dst = os.path.join(os.path.join(train_dir, cls), fname)
        shutil.copyfile(src, dst)
        
    validation_fnames = fnames[train_size:(validation_size + train_size)]
    print("Validation size(",cls,"): ", len(validation_fnames))
    for fname in validation_fnames:
        src = os.path.join(path, fname)
        dst = os.path.join(os.path.join(validation_dir, cls), fname)
        shutil.copyfile(src, dst)
        
    test_fnames = fnames[(train_size+validation_size):(validation_size + train_size +test_size)]

    print("Test size(",cls,"): ", len(test_fnames))
    for fname in test_fnames:
        src = os.path.join(path, fname)
        dst = os.path.join(os.path.join(test_dir, cls), fname)
        shutil.copyfile(src, dst)

Train size( Pepper,_bell___healthy ):  886
Validation size( Pepper,_bell___healthy ):  295
Test size( Pepper,_bell___healthy ):  295
Train size( Grape___Esca_(Black_Measles) ):  829
Validation size( Grape___Esca_(Black_Measles) ):  276
Test size( Grape___Esca_(Black_Measles) ):  276
Train size( Pepper,_bell___Bacterial_spot ):  598
Validation size( Pepper,_bell___Bacterial_spot ):  199
Test size( Pepper,_bell___Bacterial_spot ):  199
Train size( Strawberry___healthy ):  273
Validation size( Strawberry___healthy ):  91
Test size( Strawberry___healthy ):  91
Train size( Grape___Black_rot ):  708
Validation size( Grape___Black_rot ):  236
Test size( Grape___Black_rot ):  236
Train size( Corn___Common_rust ):  715
Validation size( Corn___Common_rust ):  238
Test size( Corn___Common_rust ):  238
Train size( Apple___Apple_scab ):  378
Validation size( Apple___Apple_scab ):  126
Test size( Apple___Apple_scab ):  126
Train size( Potato___healthy ):  91
Validation size( Potato___healthy ):  30


In [23]:
import torch
import os

#cuda setting
USE_CUDA = torch.cuda.is_available()
DEVICE = torch.device("cuda" if USE_CUDA else "cpu")
BATCH_SIZE = 256 
EPOCH = 30 #dataset iteration count

In [24]:
print(USE_CUDA)

True


In [25]:
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder 

#set the way how image will be preprocessed
transform_base = transforms.Compose([transforms.Resize((64,64)), transforms.ToTensor()]) 

#load image dataset from the root folder(transform option: preprocessing method)
train_dataset = ImageFolder(root='/content/drive/MyDrive/splitted/train', transform=transform_base) 
val_dataset = ImageFolder(root='/content/drive/MyDrive/splitted/val', transform=transform_base)

In [26]:
from torch.utils.data import DataLoader

#divide image dataset up into the size of minibatch
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=4)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=4)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [27]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
 
class Net(nn.Module): 
  
    def __init__(self): 
    
        super(Net, self).__init__() 
        
        #input channel: 3(color image), output channel: 32, filter size:3, padding(L,R,U,D):1 
        self.conv1 = nn.Conv2d(3, 32, 3, padding=1) 

        #first: conv, next:max pooling
        self.pool = nn.MaxPool2d(2,2)  

        #the number of input channel should be same with the number of previous output channel
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)  
        self.conv3 = nn.Conv2d(64, 64, 3, padding=1)  
        
        #image size
        #conv1 -> max pooling: 32
        #conv2 -> max pooling: 16
        #conv3 -> max pooling: 8(final size of the image)
        #Thus, 64(output filter) * 8 (width) * 8(height) = 4096
        self.fc1 = nn.Linear(4096, 512) 
        self.fc2 = nn.Linear(512, 33) 
    
    def forward(self, x):  
    
        x = self.conv1(x)
        x = F.relu(x)  
        x = self.pool(x) 
        x = F.dropout(x, p=0.25, training=self.training) 

        x = self.conv2(x)
        x = F.relu(x) 
        x = self.pool(x) 
        x = F.dropout(x, p=0.25, training=self.training)

        x = self.conv3(x) 
        x = F.relu(x) 
        x = self.pool(x) 
        x = F.dropout(x, p=0.25, training=self.training)

        #flatten dataset to 1 by 4096
        x = x.view(-1, 4096)  
        x = self.fc1(x) 
        x = F.relu(x) 
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.fc2(x) 

        return F.log_softmax(x, dim=1)  

model_base = Net().to(DEVICE)  
optimizer = optim.Adam(model_base.parameters(), lr=0.001) 

In [28]:
def train(model, train_loader, optimizer):
    model.train()  
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(DEVICE), target.to(DEVICE) 
        optimizer.zero_grad() #initialize previous gradients
        output = model(data) 
        loss = F.cross_entropy(output, target) 
        loss.backward() #back propagate to get new gradient in each parameter
        optimizer.step() #update model's parameter with the new gradient

In [29]:
def evaluate(model, test_loader):
    model.eval()  
    test_loss = 0 
    correct = 0   
    
    with torch.no_grad():#prevent parameter from updating 
        for data, target in test_loader:  
            data, target = data.to(DEVICE), target.to(DEVICE) #allocate CUDA device 
            output = model(data) 
            
            #measure loss with cross_entropy function(softmax which consists of exponential)
            #softmax--> normalize a value between 0 and 1
            #item()--> return a value from tensor type
            test_loss += F.cross_entropy(output,target, reduction='sum').item() 
            
            pred = output.max(1, keepdim=True)[1]
            
            #compare target with prediction as they have same shape after using view_as()
            correct += pred.eq(target.view_as(pred)).sum().item() 
   
    test_loss /= len(test_loader.dataset) 
    test_accuracy = 100. * correct / len(test_loader.dataset) 
    return test_loss, test_accuracy 

In [30]:
import time
import copy
 
def train_baseline(model ,train_loader, val_loader, optimizer, num_epochs = 30):
    best_acc = 0.0  
    best_model_wts = copy.deepcopy(model.state_dict()) 
 
    for epoch in range(1, num_epochs + 1):
        since = time.time()  #measure elasped time
        train(model, train_loader, optimizer) #train model with dataset and optimizer
        train_loss, train_acc = evaluate(model, train_loader)  #get train loss and accuracy
        val_loss, val_acc = evaluate(model, val_loader) #get validation loss and accuracy
        
        if val_acc > best_acc: #if validation accuary is higher than previous best one, then change the value
            best_acc = val_acc 
            best_model_wts = copy.deepcopy(model.state_dict())
        
        time_elapsed = time.time() - since 
        print('-------------- epoch {} ----------------'.format(epoch))
        print('train Loss: {:.4f}, Accuracy: {:.2f}%'.format(train_loss, train_acc))   
        print('val Loss: {:.4f}, Accuracy: {:.2f}%'.format(val_loss, val_acc))
        print('Completed in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60)) 
    model.load_state_dict(best_model_wts)  
    return model
 

base = train_baseline(model_base, train_loader, val_loader, optimizer, EPOCH)  	 #(16)
torch.save(base,'baseline.pt')

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


-------------- epoch 1 ----------------
train Loss: 1.6174, Accuracy: 54.79%
val Loss: 1.6486, Accuracy: 53.49%
Completed in 2m 23s
-------------- epoch 2 ----------------
train Loss: 0.9491, Accuracy: 72.12%
val Loss: 0.9908, Accuracy: 70.46%
Completed in 2m 18s
-------------- epoch 3 ----------------
train Loss: 0.7374, Accuracy: 77.26%
val Loss: 0.7948, Accuracy: 75.45%
Completed in 2m 18s
-------------- epoch 4 ----------------
train Loss: 0.5954, Accuracy: 81.62%
val Loss: 0.6567, Accuracy: 80.00%
Completed in 2m 17s
-------------- epoch 5 ----------------
train Loss: 0.5056, Accuracy: 84.31%
val Loss: 0.5709, Accuracy: 81.81%
Completed in 2m 20s
-------------- epoch 6 ----------------
train Loss: 0.4320, Accuracy: 86.38%
val Loss: 0.5074, Accuracy: 83.61%
Completed in 2m 19s
-------------- epoch 7 ----------------
train Loss: 0.3856, Accuracy: 87.99%
val Loss: 0.4626, Accuracy: 85.25%
Completed in 2m 20s
-------------- epoch 8 ----------------
train Loss: 0.4326, Accuracy: 85.44%

In [31]:
data_transforms = {
    'train': transforms.Compose([transforms.Resize([64,64]), 
        transforms.RandomHorizontalFlip(), transforms.RandomVerticalFlip(),  
        transforms.RandomCrop(52), transforms.ToTensor(), 
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]) ]),
    
    'val': transforms.Compose([transforms.Resize([64,64]),  
        transforms.RandomCrop(52), transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]) ])
}

In [32]:
data_dir = '/content/drive/MyDrive/splitted' 
image_datasets = {x: ImageFolder(root=os.path.join(data_dir, x), transform=data_transforms[x]) for x in ['train', 'val']} 
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=BATCH_SIZE, shuffle=True, num_workers=4) for x in ['train', 'val']} 
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}

class_names = image_datasets['train'].classes

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [33]:
from torchvision import models
 
resnet = models.resnet50(pretrained=True)  #True: use pretrained parameters, False: parameters will be set randomly
num_ftrs = resnet.fc.in_features   
resnet.fc = nn.Linear(num_ftrs, 33) 
resnet = resnet.to(DEVICE)
 
criterion = nn.CrossEntropyLoss() 

#among parameters in ResNet model, if a parameter is set as true to update,
#update the parameter with Adam optimizer.
optimizer_ft = optim.Adam(filter(lambda p: p.requires_grad, resnet.parameters()), lr=0.001)
 
from torch.optim import lr_scheduler
#reduce learning rate as 7 epoch has passed.
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1) 

Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth


  0%|          | 0.00/97.8M [00:00<?, ?B/s]

In [34]:
ct = 0 
#freeze layers before 6th(total 10 layers)
#1~5 layers: freeze
#6~10: don't freeze
for child in resnet.children():
    ct += 1  
    #print(ct) --> check current layer if uncommented  
    if ct < 6: 
        for param in child.parameters():
            param.requires_grad = False

In [35]:
def train_resnet(model, criterion, optimizer, scheduler, num_epochs=25):

    best_model_wts = copy.deepcopy(model.state_dict())  
    best_acc = 0.0  
    
    for epoch in range(num_epochs):
        print('-------------- epoch {} ----------------'.format(epoch+1)) 
        since = time.time()                                     
        for phase in ['train', 'val']: 
            if phase == 'train': 
                model.train() 
            else:
                model.eval()     
 
            running_loss = 0.0  
            running_corrects = 0  
 
            
            for inputs, labels in dataloaders[phase]: 
                inputs = inputs.to(DEVICE)  
                labels = labels.to(DEVICE)  
                
                optimizer.zero_grad() 
                
                with torch.set_grad_enabled(phase == 'train'):  
                    outputs = model(inputs)  
                    _, preds = torch.max(outputs, 1) 
                    loss = criterion(outputs, labels)  
    
                    if phase == 'train':   
                        loss.backward()
                        optimizer.step()
 
                running_loss += loss.item() * inputs.size(0)  
                running_corrects += torch.sum(preds == labels.data)  
            if phase == 'train':  
                scheduler.step()
 
            epoch_loss = running_loss/dataset_sizes[phase]  
            epoch_acc = running_corrects.double()/dataset_sizes[phase]  
 
            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc)) 
 
          
            if phase == 'val' and epoch_acc > best_acc: 
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
 
        time_elapsed = time.time() - since  
        print('Completed in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))
 
    model.load_state_dict(best_model_wts) 

    return model

In [36]:
model_resnet50 = train_resnet(resnet, criterion, optimizer_ft, exp_lr_scheduler, num_epochs=EPOCH) 

torch.save(model_resnet50, 'resnet50.pt')

-------------- epoch 1 ----------------


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


train Loss: 0.5849 Acc: 0.8249
val Loss: 0.3381 Acc: 0.8975
Completed in 1m 39s
-------------- epoch 2 ----------------
train Loss: 0.2262 Acc: 0.9273
val Loss: 0.2530 Acc: 0.9179
Completed in 1m 39s
-------------- epoch 3 ----------------
train Loss: 0.1720 Acc: 0.9451
val Loss: 0.1812 Acc: 0.9462
Completed in 1m 39s
-------------- epoch 4 ----------------
train Loss: 0.1402 Acc: 0.9555
val Loss: 0.2001 Acc: 0.9387
Completed in 1m 38s
-------------- epoch 5 ----------------
train Loss: 0.1155 Acc: 0.9631
val Loss: 0.1318 Acc: 0.9574
Completed in 1m 37s
-------------- epoch 6 ----------------
train Loss: 0.0933 Acc: 0.9689
val Loss: 0.1312 Acc: 0.9604
Completed in 1m 39s
-------------- epoch 7 ----------------
train Loss: 0.0911 Acc: 0.9700
val Loss: 0.1487 Acc: 0.9592
Completed in 1m 40s
-------------- epoch 8 ----------------
train Loss: 0.0521 Acc: 0.9834
val Loss: 0.0484 Acc: 0.9850
Completed in 1m 40s
-------------- epoch 9 ----------------
train Loss: 0.0289 Acc: 0.9911
val Loss:

In [38]:
transform_base = transforms.Compose([transforms.Resize([64,64]),transforms.ToTensor()])
test_base = ImageFolder(root='/content/drive/MyDrive/splitted/test',transform=transform_base)  
test_loader_base = torch.utils.data.DataLoader(test_base, batch_size=BATCH_SIZE, shuffle=True, num_workers=4)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [39]:
transform_resNet = transforms.Compose([
        transforms.Resize([64,64]),  
        transforms.RandomCrop(52),  
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]) 
        #first list: average value while applying to normalization in each channel(RGB)
        #second list: standard deviation while applying to normalization in each channel(RGB)
    ])
    
test_resNet = ImageFolder(root='/content/drive/MyDrive/splitted/test', transform=transform_resNet) 
test_loader_resNet = torch.utils.data.DataLoader(test_resNet, batch_size=BATCH_SIZE, shuffle=True, num_workers=4)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [40]:
baseline=torch.load('baseline.pt') 
baseline.eval()  
test_loss, test_accuracy = evaluate(baseline, test_loader_base)

print('baseline test acc:  ', test_accuracy)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


baseline test acc:   93.50356740518212


In [41]:
resnet50=torch.load('resnet50.pt') 
resnet50.eval()  
test_loss, test_accuracy = evaluate(resnet50, test_loader_resNet)

print('ResNet test acc:  ', test_accuracy)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


ResNet test acc:   98.84841657278758
